In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()


In [4]:
len(words)

32033

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [6]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # rolling window

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [41]:
# MLP revisited
n_embed = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embed), generator=g) # embeddings table: it basically converts any given input character into a n_embed dimensional vector (later improvement - 10 dimensional)
# neural network first layer (hidden layer)
# kaiming init, torch.nn.init for explanation
W1 = torch.randn((n_embed * block_size, n_hidden), generator=g) * (5/3)/((n_embed* block_size)**0.5) # weights, we scaled it up from 100 to 300 neurons
b1 = torch.randn(n_hidden, generator=g) * 0.01 # biases, small number to avoid dead neurons
# neural network second layer (output layer)
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.01 # weights, initialize to small numbers to avoid the hockey stick effect in the beginning
b2 = torch.randn(vocab_size, generator=g) * 0 # biases are initially zero to make distribution more uniform
parameters = [C, W1, b1, W2, b2]
for p in parameters:
  p.requires_grad = True

In [42]:
# training
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (32, ))
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X, Y

  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  hpreact = embcat @ W1 + b1 # hidden layer pre-activation
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function

  # backward pass
  for p in parameters:
    p.grad = None 
  loss.backward()

  # update
  lr = 0.1 if i < (max_steps / 2) else 0.01
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 10000 == 0:
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())


      0/ 200000: 3.3158
  10000/ 200000: 1.8832
  20000/ 200000: 2.1009
  30000/ 200000: 2.0953
  40000/ 200000: 2.2420
  50000/ 200000: 2.4566
  60000/ 200000: 2.3871
  70000/ 200000: 2.0300
  80000/ 200000: 2.4022
  90000/ 200000: 1.7395
 100000/ 200000: 2.1754
 110000/ 200000: 1.9768
 120000/ 200000: 1.8832
 130000/ 200000: 2.3117
 140000/ 200000: 2.1822
 150000/ 200000: 1.9946
 160000/ 200000: 1.9916
 170000/ 200000: 2.8484
 180000/ 200000: 2.1971
 190000/ 200000: 2.5573


In [43]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split_label):
  x, y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte)
  }[split_label]

  emb = C[x] # (N, block_size, n_embed)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embed)
  h = torch.tanh(embcat @ W1 + b1) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size
  loss = F.cross_entropy(logits, y)
  print(split_label, loss.item())

split_loss('train')
split_loss('val')

train 2.0377724170684814
val 2.1088626384735107


In [44]:
# sample from the model
g = torch.Generator().manual_seed(2147483647  +10)
for _ in range(20):
  out = []
  context = [0] * block_size # initialize with ...

  while True:
    # forward pass the neural net
    emb = C[torch.tensor([context])] # (1, block_size, dn_embed
    h = torch.tanh(emb.view(1, -1) @ W1 + b1)
    logits = h @ W2 + b2
    probs = F.softmax(logits, dim=1)
    # sample from the distribution
    ix = torch.multinomial(probs, num_samples=1, generator=g).item()
    # shift the context window and track the samples
    context = context[1:] + [ix]
    out.append(ix)
    # if we sample the special '.' break, word ends here
    if ix == 0:
      break

  print(''.join(itos[i] for i in out))

carmaveah.
jhovi.
kemrix.
taty.
salessa.
jazzie.
faverynciereni.
nellara.
chaihron.
leigh.
ham.
jory.
quintis.
lilea.
jadiquinte.
madiaryn.
kai.
evedhuan.
emmedi.
abette.
